<a href="https://colab.research.google.com/github/megmarv/PsychoAI-/blob/Image-Retrieval/Custom_Made_Face_Recognition_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python tensorflow numpy matplotlib


Face Extraction and Classification Code

Dataset Preparation

In [ ]:
import os
import cv2
import numpy as np

# Function to load an image and its corresponding bounding boxes
def load_image_and_annotations(image_path, annotation_path, img_size=(224, 224)):
    # Load image
    img = cv2.imread(image_path)
    img = cv2.resize(img, img_size)
    img = img.astype('float32') / 255.0  # Normalize image

    # Load annotations (bounding boxes)
    bboxes = []
    with open(annotation_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            bbox = line.strip().split()
            if len(bbox) >= 4:
                bbox = [float(coord) for coord in bbox[:4]]
                bboxes.append(bbox)

    return img, np.array(bboxes)

# Function to load dataset from AFW folder
def load_afw_dataset(image_dir, annotation_dir, img_size=(224, 224)):
    images = []
    bboxes = []

    for img_name in os.listdir(image_dir):
        if img_name.endswith('.jpg'):
            img_path = os.path.join(image_dir, img_name)
            annotation_path = os.path.join(annotation_dir, img_name.replace('.jpg', '.txt'))

            # Load image and annotations
            img, bbox = load_image_and_annotations(img_path, annotation_path, img_size)

            images.append(img)
            bboxes.append(bbox)

    images = np.array(images)
    bboxes = np.array(bboxes)

    return images, bboxes

# Example usage
dataset_dir = 'path_to_afw_dataset'  # Update with path to your AFW dataset
image_dir = os.path.join(dataset_dir, 'images')
annotation_dir = os.path.join(dataset_dir, 'annotations')

# Load the dataset
images, bboxes = load_afw_dataset(image_dir, annotation_dir)

print(f"Loaded {len(images)} images with bounding boxes.")


Model Creation

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

# Custom CNN for Face Detection (Bounding Box Prediction)
def build_face_detection_model(input_shape):
    model = models.Sequential([
        # Feature Extraction Layers
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        # Bounding Box Prediction Layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(4, activation='sigmoid')  # 4 values: x, y, width, height
    ])
    return model

# Compile model
def compile_model(model):
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Model summary
input_shape = (224, 224, 3)  # Resize image to (224, 224)
model = build_face_detection_model(input_shape)
model = compile_model(model)
model.summary()


Model Training

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, bboxes, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Model Testing and Evaluation

In [ ]:
# Evaluate model on test data
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Test the model on a single image
def predict_face(model, image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, target_size)
    img_normalized = img_resized.astype('float32') / 255.0
    img_input = np.expand_dims(img_normalized, axis=0)  # Add batch dimension

    # Predict bounding box for face
    bbox = model.predict(img_input)[0]

    # Scale bounding box back to original image size
    h, w, _ = img.shape
    bbox = [int(coord * dimension) for coord, dimension in zip(bbox, [w, h, w, h])]

    # Extract face using the predicted bounding box
    x, y, w, h = bbox
    face = img[y:y+h, x:x+w]

    return face, bbox

# Example usage: Test on a single image
image_path = 'path_to_image.jpg'
face, bbox = predict_face(model, image_path)

# Show the extracted face
if face is not None:
    cv2.imshow('Extracted Face', face)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
